In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import Future 
import csv
import time

In [2]:
start_url='https://arxiv.org/'

In [3]:
def get_urls(url):
    """ Returns a list of urls on the page """
    str='https://arxiv.org'
    r = requests.get(url)
    soup=BeautifulSoup(r.text, 'html.parser')
    href=[t['href'] for t in soup.select('div#content li a')]
    href_goal=[t for t in href if 'list' in t]
    URLs=[str + t for t in href_goal]
    return URLs  

In [4]:
def load_url(url):
    r=requests.get(url)
    soup=BeautifulSoup(r.text, 'html.parser')
    """ Returns list of titles of a url on the page """
    titles=[t.get_text() for t in soup.select('.list-title.mathjax')]
    Titles=[t.splitlines()[1] for t in titles]
    """ Returns list of authors of a url on the page """
    authors=[t.get_text() for t in soup.select('.list-authors')]
    Authors=[t.splitlines()[2:len(authors)] for t in authors]
    """ Returns list of subjects on the page """
    Subjects=[t.get_text() for t in soup.select('.primary-subject')]
    """ Returns list of links of abstracts and full_texts on the page """
    str='https://arxiv.org'
    pdf_abs=[t['href'] for t in soup.select('.list-identifier a')]
    pdf=[t for t in pdf_abs if '/pdf/' in t]
    PDF=[str + t for t in pdf]
    pdf=[t for t in pdf_abs if '/abs/' in t]
    Abs=[str + t for t in pdf]
    """ Returns list of comments on the page """
    comments=[t.get_text() for t in soup.select('.meta div.list-comments.mathjax')]
    Comments=[t.splitlines()[1] for t in comments]
    

    Output = {
    'Subject:':Subjects,   
    'Title:': Titles,
    'Authors:':Authors,
    'Comments:':Comments,
    'Abstract_url:':Abs,
    'Full text_url:':PDF
                }
    Dframe = pd.DataFrame.from_dict(Output, orient="index")
    
    return Dframe



with ThreadPoolExecutor(185) as pool:
    start=time.time()
    
    futures1 = [ pool.submit(load_url, url) for url in get_urls(start_url)]
    data = [] 
    for future in concurrent.futures.as_completed(futures1):
        scraped_res=future.result()
        data.append(scraped_res)
    end = time.time()
              
    print("Time Taken: {:.6f}s".format(end-start))    

Time Taken: 34.088391s


In [ ]:
data_concat=pd.concat(data, axis=1, ignore_index=True)
data_final=data_concat.T
""" Save results """
data_final.to_csv(r'C:\Users\Rodrigo\Box Sync\phyton GSE\scraping\exercise\data_assignment_gse.csv', header=True, 
              index = True, encoding='utf-8')